In [4]:
import pandas as pd

creditData = pd.read_csv("credit_card.csv")
approvalData = pd.read_csv("credit_card_label.csv")
print(approvalData)

       Ind_ID  label
0     5008827      1
1     5009744      1
2     5009746      1
3     5009749      1
4     5009752      1
...       ...    ...
1543  5028645      0
1544  5023655      0
1545  5115992      0
1546  5118219      0
1547  5053790      0

[1548 rows x 2 columns]
